In [ ]:
from qiskit_dynamics import Solver, DynamicsBackend
from qiskit_dynamics.backend import default_experiment_result_function
from qiskit_dynamics.array import Array
import jax
from qiskit.providers.fake_provider import *
from qiskit import pulse
from qiskit import IBMQ

In [ ]:
from qiskit_ibm_provider import IBMProvider

IBMQ.save_account('cc2b3dd2f2068c2e5e4dcb862c7055b06b677a541579df46cf916303e2e2264e1999e9cd2d4b2e1ea6da15485a712aa1444bae52a93c0eb6d684bd02f9107e6c')
IBMQ.load_account()

provider = IBMQ.get_provider('ibm-q')
available_backends = provider.backends()
print("Backends disponibili:")
for backend in available_backends:
    print(backend.name())

In [ ]:
'''from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper
import numpy as np
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver

gate_backend =FakeManila()
print("Porte logiche supportate dal backend:", gate_backend.configuration().basis_gates)
gate_backend.configuration().hamiltonian['qub'] = {'0': 2,'1': 2,'2': 2,'3': 2,'4': 2}
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")
Array.set_default_backend("jax")
pulse_backend = DynamicsBackend.from_backend(gate_backend, evaluation_mode="sparse")
solver_options = {"method": "jax_odeint", "atol": 1e-6, "rtol": 1e-8}
pulse_backend.set_options(solver_options=solver_options)
pulse_backend.configuration = lambda: gate_backend.configuration()'''

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper
import numpy as np
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.providers.ibmq import IBMQBackend

gate_backend = provider.get_backend('ibm_osaka')
print("Porte logiche supportate dal backend ibm_osaka:", gate_backend.configuration().basis_gates)
#gate_backend.qubit_properties(0)

In [ ]:
#gate_backend.qubit_properties(1)

In [ ]:
import numpy as np

dim = 3

v0 = 4822503916.367367 #5091732026.655052 #4.86e9
anharm0 = -308966137.92557317 #-306417871.3208025 #-0.32e9
r0 = 0.22e9

v1 = 4772686708.526122 #4970568360.164501 #4.97e9
anharm1 = -310221076.1952548 #-307611847.30564153 # -0.32e9
r1 = 0.26e9

J = 0.002e9

a = np.diag(np.sqrt(np.arange(1, dim)), 1)
adag = np.diag(np.sqrt(np.arange(1, dim)), -1)
N = np.diag(np.arange(dim))

ident = np.eye(dim, dtype=complex)
full_ident = np.eye(dim**2, dtype=complex)

N0 = np.kron(ident, N)
N1 = np.kron(N, ident)

a0 = np.kron(ident, a)
a1 = np.kron(a, ident)

a0dag = np.kron(ident, adag)
a1dag = np.kron(adag, ident)


static_ham0 = 2 * np.pi * v0 * N0 + np.pi * anharm0 * N0 * (N0 - full_ident)
static_ham1 = 2 * np.pi * v1 * N1 + np.pi * anharm1 * N1 * (N1 - full_ident)

static_ham_full = static_ham0 + static_ham1 + 2 * np.pi * J * ((a0 + a0dag) @ (a1 + a1dag))

drive_op0 = 2 * np.pi * r0 * (a0 + a0dag)
drive_op1 = 2 * np.pi * r1 * (a1 + a1dag)

In [ ]:
from qiskit_dynamics import Solver

# build solver
dt = 1/4.5e9

solver = Solver(
    static_hamiltonian=static_ham_full,
    hamiltonian_operators=[drive_op0, drive_op1, drive_op0, drive_op1],
    rotating_frame=static_ham_full,
    hamiltonian_channels=["d0", "d1", "u0", "u1"],
    channel_carrier_freqs={"d0": v0, "d1": v1, "u0": v1, "u1": v0},
    dt=dt,
)

In [ ]:
from qiskit_dynamics import DynamicsBackend

# Consistent solver option to use throughout notebook
solver_options = {"method": "RK45", "atol": 1e-6, "rtol": 1e-8}

pulse_backend = DynamicsBackend(
    solver=solver,
    subsystem_dims=[dim, dim], # for computing measurement data
    solver_options=solver_options, # to be used every time run is called
)

In [ ]:
import qiskit_nature
qiskit_nature.settings.use_pauli_sum_op = False  # pylint: disable=undefined-variable

In [ ]:
from qiskit_nature.second_q.transformers import FreezeCoreTransformer,ActiveSpaceTransformer
import re
my_dict = {}

def get_qubit_op(dist):
    # Define Molecule
    ultra_simplified_ala_string = f"""
    H 0.0 0.0 0.0
    H 0.0 0.0 {dist}
    """
    
    driver = PySCFDriver(
        atom=ultra_simplified_ala_string.strip(),
        basis='sto3g',
        unit=DistanceUnit.ANGSTROM
    )
    qmolecule = driver.run()
    '''
    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(qmolecule)

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals

    mapper = ParityMapper(num_particles=num_particles)
    qubit_op = mapper.map(problem.second_q_ops()[0])
    #print(qubit_op)'''
    num_active_electrons = 2
    num_active_orbitals = 2
    as_transformer = ActiveSpaceTransformer(num_active_electrons,num_active_orbitals)
    problem = as_transformer.transform(qmolecule)
    #print(problem.nuclear_repulsion_energy)
    mapper = ParityMapper(num_particles = problem.num_particles)
    qubit_op = mapper.map(problem.second_q_ops()[0])
    
    for pauli in qubit_op:
        #print(pauli)
        str_info = pauli.__str__()
        #print(f"Rappresentazione della stringa: {str_info}")
    
        # Estrai i coefficienti
        coeffs_start = str_info.find("coeffs=[") + len("coeffs=[")
        coeffs_end = str_info.find("]", coeffs_start)
        coeffs_str_complex = complex(str_info[coeffs_start:coeffs_end].strip())
        coeffs_str = coeffs_str_complex.real
        #print(coeffs_str)
        
        # Estrai gli operatori come stringa
        operators_start = str_info.find("[") + len("[")
        operators_end = str_info.find("])", operators_start)
        operators_str = str_info[operators_start:operators_end].strip()
        label_start = operators_str.find("'") + 1
        label_end = operators_str.find("'", label_start)
        label = operators_str[label_start:label_end]
        #print(label)
    
        '''# Utilizza espressioni regolari per estrarre la parte reale e immaginaria dei coefficienti
        coeffs_match = re.findall(r"(-?\d+\.\d+)([+-]\d+\.\d+j)?", coeffs_str)
    
        # Inizializza i coefficienti come float
        real_coeffs = [float(match[0]) for match in coeffs_match]
        imag_coeffs = [float(match[1]) if match[1] else 0.0 for match in coeffs_match]
    
        # Somma i coefficienti complessi
        complex_coeff = sum(complex(real, imag) for real, imag in zip(real_coeffs, imag_coeffs))'''
    
        # Aggiungi il coefficiente all'operatore nel dizionario
        #current_coeff = my_dict.get(operators_str, 0.0)
        #my_dict[operators_str] = coeffs_str
        my_dict[label] = coeffs_str
        
        
    return qubit_op, my_dict,problem,problem.nuclear_repulsion_energy

In [ ]:
from qiskit_nature import settings
settings.use_pauli_sum_op = False

In [ ]:
from qiskit.pulse import Schedule, GaussianSquare, Drag, Delay, Play, ControlChannel, DriveChannel

def drag_pulse(backend, amp, angle):
  backend_defaults = backend.defaults()
  inst_sched_map = backend_defaults.instruction_schedule_map
  x_pulse = inst_sched_map.get('x', (0)).filter(channels = [DriveChannel(0)], instruction_types=[Play]).instructions[0][1].pulse
  duration_parameter = x_pulse.parameters['duration']
  sigma_parameter = x_pulse.parameters['sigma']
  beta_parameter = x_pulse.parameters['beta']
  pulse1 = Drag(duration=duration_parameter, sigma=sigma_parameter, beta=beta_parameter, amp=amp, angle=angle)
  return pulse1

In [ ]:
def ecr_pulse(backend, amp, angle, duration):
  backend_defaults = backend.defaults()
  inst_sched_map = backend_defaults.instruction_schedule_map
  ecr_pulse = inst_sched_map.get('ecr', (1, 0)).filter(channels = [ControlChannel(0)], instruction_types=[Play]).instructions[0][1].pulse
  ecr_params = {}
  ecr_params['duration'] = ecr_pulse.parameters['duration']
  ecr_params['amp'] = ecr_pulse.parameters['amp']
  ecr_params['angle'] = ecr_pulse.parameters['angle']
  ecr_params['sigma'] = ecr_pulse.parameters['sigma']
  ecr_params['width'] = ecr_pulse.parameters['width']
  ecr_risefall = (ecr_params['duration'] - ecr_params['width']) / (2 * ecr_params['sigma'])
  angle_parameter = angle
  duration_parameter =  duration
  sigma_parameter = ecr_pulse.parameters['sigma']
  width_parameter = int(duration_parameter - 2 * ecr_risefall * ecr_params['sigma'])
  #declare pulse parameters and build GaussianSquare pulse
  pulse1 = GaussianSquare(duration = duration_parameter, amp = amp, angle = angle_parameter, sigma = sigma_parameter, width=width_parameter)
  return pulse1
'''from qiskit import QuantumCircuit,pulse
from qiskit.circuit.library import ECRGate, RZGate, SXGate
from qiskit.pulse import GaussianSquare, ControlChannel
from qiskit import QuantumRegister
from qiskit.pulse import Schedule, GaussianSquare, Drag, Delay, Play, ControlChannel, DriveChannel
from qiskit.pulse import Waveform, ScalableSymbolicPulse
from qiskit.pulse import library

def cr_pulse(backend, amp, angle, duration):
    backend_defaults = backend.defaults()
    inst_sched_map = backend_defaults.instruction_schedule_map
    x1 = inst_sched_map.get('x', (1)).filter(channels = [DriveChannel(1)], instruction_types=[Play]).instructions[0][1].pulse
    id = library.Constant(duration=duration, amp=amp)
    id0 = pulse.Play(id, pulse.DriveChannel(0)).instructions[0][1].pulse
    cr_pulse = x1 #np.convolve(x1, id0, mode='full')
    cr_params = {}
    cr_params = {
        'duration': cr_pulse.duration,
        'amp': cr_pulse.amp,
        'angle': cr_pulse.angle,
        'sigma': cr_pulse.sigma,
        #'width': cr_pulse.width
    }
    #cr_risefall = (cr_params['duration'] - cr_params['width']) / (2 * cr_params['sigma'])
    cr_risefall = (cr_params['duration']) / (2 * cr_params['sigma'])
    angle_parameter = angle
    duration_parameter = duration
    sigma_parameter = cr_pulse.sigma
    width_parameter = int(duration_parameter - 2 * cr_risefall * cr_params['sigma'])
    pulse1 = GaussianSquare(duration=duration_parameter, amp=amp, angle=angle_parameter,
                            sigma=sigma_parameter, width=duration) #width_parameter)
    return pulse1'''


In [ ]:
'''def HE_pulse(backend, amp, angle, width):
    with pulse.build(backend) as my_program1:
      # layer 1
      sched_list = []
      with pulse.build(backend) as sched1:
          qubits = (0,1,2,3)
          for i in range(4):
              pulse.play(drag_pulse(backend, amp[i], angle[i]), DriveChannel(qubits[i]))
      sched_list.append(sched1)

      with pulse.build(backend) as sched2:
          uchan = pulse.control_channels(0, 1)[0]
          pulse.play(cr_pulse(backend,amp[4], angle[4], width[0]), uchan)
      sched_list.append(sched2)


      with pulse.build(backend) as sched4:
          uchan = pulse.control_channels(1, 2)[0]
          pulse.play(cr_pulse(backend, amp[5], angle[5], width[1]), uchan)
      sched_list.append(sched4)

      with pulse.build(backend) as sched6:
          uchan = pulse.control_channels(2,3)[0]
          pulse.play(cr_pulse(backend, amp[6],angle[6], width[2]), uchan)
      sched_list.append(sched6)


      with pulse.build(backend) as my_program:
        with pulse.transpiler_settings(initial_layout= [0,1,2,3]):
          with pulse.align_sequential():
              for sched in sched_list:
                  pulse.call(sched)

    return my_program'''

def HE_pulse(backend, amp, angle, width):
    with pulse.build(backend) as my_program1:
        # layer 1
        sched_list = []
        with pulse.build(backend) as sched1:
            qubits = (1, 0)  # Imposta i qubit su cui operare
            for i in range(2):
                pulse.play(drag_pulse(backend, amp[i], angle[i]), DriveChannel(qubits[i]))
        sched_list.append(sched1)

        with pulse.build(backend) as sched2:
            uchan = pulse.control_channels(1, 0)[0] 
            pulse.play(ecr_pulse(backend, amp[0], angle[2], width[0]), uchan)
        sched_list.append(sched2)

        with pulse.build(backend) as sched4:
            uchan = pulse.control_channels(1, 0)[0]  
            pulse.play(ecr_pulse(backend, amp[1], angle[3], width[0]), uchan)
        sched_list.append(sched4)

        with pulse.build(backend) as my_program:
            with pulse.transpiler_settings(initial_layout=[0, 1]):  # Specifica la disposizione dei qubit
                with pulse.align_sequential():
                    for sched in sched_list:
                        pulse.call(sched)

    return my_program

In [ ]:
import copy
import qiskit
from scipy.optimize import minimize, LinearConstraint
from qiskit.pulse.transforms import block_to_schedule
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector

def measurement_pauli(prepulse, pauli_string, backend, n_qubit):
    with pulse.build(backend) as pulse_measure:
        pulse.call(copy.deepcopy(prepulse))
        for ind,pauli in enumerate(pauli_string):
            if(pauli=='X'):
                pulse.u2(0, np.pi, ind)
            if(pauli=='Y'):
                pulse.u2(0, np.pi/2, ind)
        for qubit in range(n_qubit):
            pulse.barrier(qubit)
        pulse.measure(range(n_qubit))
    return pulse_measure

def n_one(bitstring, key):
    results = 0
    for ind,b in enumerate(reversed(bitstring)):
        if((b=='1')&(key[ind]!='I')):
            results+=1
    return results

def expectation_value(counts,shots,key):
    results = 0
    for bitstring in counts:
        if(n_one(bitstring, key)%2==1):
            results -= counts[bitstring]/shots
        else:
            results += counts[bitstring]/shots
    return results

def run_pulse_sim(meas_pulse, key, pulse_backend, backend, n_shot):
    #job = pulse_backend.run(meas_pulse, shots=n_shot)
    job = backend.run(meas_pulse)
    job_monitor(job)
    counts = job.result().get_counts()
    expectation = expectation_value(counts,n_shot,key)
    return expectation

def gen_LC_vqe(parameters):
    lb = np.zeros(parameters)
    ub = np.ones(parameters)
    LC = (LinearConstraint(np.eye(parameters),lb,ub,keep_feasible=False))
    return LC

In [ ]:
from qiskit.tools.monitor import job_monitor

def vqe_one(prepulse,n_qubit,n_shot,pulse_backend, backend,key,value):
    all_Is = True
    for key_ele in key:
        if(key_ele!='I'):
            all_Is = False
    if(all_Is):
        return value
    meas_pulse = measurement_pauli(prepulse=prepulse, pauli_string=key, backend=backend, n_qubit=n_qubit)
    return value*run_pulse_sim(meas_pulse, key, pulse_backend, backend, n_shot)

In [ ]:
from qiskit.visualization.pulse_v2 import draw
from qiskit import schedule
def vqe(params,pauli_dict,pulse_backend, backend,n_qubit,n_shot):
    print("params in def chemistry in vqe.py: ", params)
    # assert(len(params)%2==0)
    width_len = int(len(params)-1*(n_qubit-1))
    split_ind = int(width_len/3)
    amp = np.array(params[:split_ind])
    angle = np.array(params[split_ind:width_len])*np.pi*2
    width_1 = (np.array(params[width_len:]))
    num_items = (1024 - 256) // 16 + 1
    width_norm = (width_1 - 256) / (1024 - 256)
    width_norm = np.clip(width_norm, 0, 1)
    width = (np.round(width_norm * (num_items - 1)) * 16 + 256).astype(int) 
    amp = amp.tolist()
    angle = angle.tolist()
    width = width.tolist()
    keys = [key for key in pauli_dict]
    values = [pauli_dict[key] for key in pauli_dict]
    expect_values = []

    for key, value in zip(keys, values):
        prepulse = HE_pulse(backend, amp, angle, width)
        sched = qiskit.pulse.transforms.block_to_schedule(prepulse)
        #print(sched)
        expect = vqe_one(prepulse, n_qubit, n_shot, pulse_backend, backend, key, value)
        expect_values.append(expect)
    print("E for cur_iter: ",sum(expect_values))
    return sum(expect_values)

In [ ]:
from qiskit.algorithms.optimizers import SPSA, SLSQP, COBYLA
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_aer.primitives import Estimator

def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

distances = np.arange(0.735, 0.740, 0.1)
exact_energies = []
pvqe_energies_1 = []
iterazioni = []
errore = []

n_qubit = 2
parameters = 7
np.random.seed(99999999)
LC = gen_LC_vqe(parameters)
params = np.zeros(parameters)
'''params = [ 1.40654244e-01 , 3.07627653e-01 , 1.62359289e-03 , 1.30853421e-01,
  1.56228238e-01 ,-2.32934060e-20 , 1.18663376e-03]
'''
n_shot = 2048
optimizer = 'COBYLA'
noiseless_estimator = Estimator(approximation=True)

#print(gate_backend)
#print(pulse_backend)

for dist in distances:
    #params = np.zeros(parameters)
    #params = np.random.rand(parameters)
    (qubit_op,my_dict,problem,REPULSION_ENERGYproblem) = get_qubit_op(dist)
    result = exact_solver(qubit_op, problem)
    exact_energies.append(result.total_energies[0].real)
    #print('distanza:', dist)
    vqe_res = minimize(vqe, params, args=(my_dict, pulse_backend, gate_backend, n_qubit, n_shot),
                       method=optimizer, constraints=LC,options={'rhobeg':0.2,'maxiter':70})
    pvqe_energies_1.append(vqe_res.fun+ REPULSION_ENERGYproblem)
    err = abs(12/100 * vqe_res.fun)
    errore.append(err)
    print('distanza:',dist)
    print('pulse_VQE:',vqe_res.fun + REPULSION_ENERGYproblem,'+/-',err)
    print('esatto:',result.total_energies[0].real)
    #print(my_dict)
'''
pvqe_energies_2 = []
params = [0.1, 0.0 , 0.,  0. , 0. , 0. , 0.,]
for dist in distances:
    #params = np.zeros(parameters)
    #params = np.random.rand(parameters)
    (qubit_op,my_dict,problem,REPULSION_ENERGYproblem) = get_qubit_op(dist)
    result = exact_solver(qubit_op, problem)
    #exact_energies.append(result.total_energies[0].real)
    #print('distanza:', dist)
    vqe_res = minimize(vqe, params, args=(my_dict, pulse_backend, gate_backend, n_qubit, n_shot),
                       method=optimizer, constraints=LC,options={'rhobeg':0.1,'maxiter':0})
    #print('The optimized loss func value: {}'.format(vqe_res.fun))
    pvqe_energies_2.append(vqe_res.fun+ REPULSION_ENERGYproblem)
    err = abs(12/100 * vqe_res.fun)
    errore.append(err)
    print('distanza:',dist)
    print('pulse_VQE:',vqe_res.fun + REPULSION_ENERGYproblem,'+/-',err)
    print('esatto:',result.total_energies[0].real)
    #print(my_dict)

pvqe_energies_3 = []
params = [ 5.77602736e-02,  1.29297911e-01,  1.76817691e-17, -3.46944695e-18,
  1.28949827e-01,  0.00000000e+00,  0.00000000e+00]
for dist in distances:
    #params = np.zeros(parameters)
    #params = np.random.rand(parameters)
    (qubit_op,my_dict,problem,REPULSION_ENERGYproblem) = get_qubit_op(dist)
    result = exact_solver(qubit_op, problem)
    #exact_energies.append(result.total_energies[0].real)
    #print('distanza:', dist)
    vqe_res = minimize(vqe, params, args=(my_dict, pulse_backend, gate_backend, n_qubit, n_shot),
                       method=optimizer, constraints=LC,options={'rhobeg':0.1,'maxiter':0})
    #print('The optimized loss func value: {}'.format(vqe_res.fun))
    pvqe_energies_3.append(vqe_res.fun+ REPULSION_ENERGYproblem)
    err = abs(12/100 * vqe_res.fun)
    errore.append(err)
    print('distanza:',dist)
    print('pulse_VQE:',vqe_res.fun + REPULSION_ENERGYproblem,'+/-',err)
    print('esatto:',result.total_energies[0].real)

pvqe_energies_4 = []
params = [ 8.673617E-20,   1.219578E-01,   1.902411E-17,   3.749686E-02,   1.599968E-01,
       0.000000E+00,   0.000000E+00]
for dist in distances:
    #params = np.zeros(parameters)
    #params = np.random.rand(parameters)
    (qubit_op,my_dict,problem,REPULSION_ENERGYproblem) = get_qubit_op(dist)
    result = exact_solver(qubit_op, problem)
    #exact_energies.append(result.total_energies[0].real)
    #print('distanza:', dist)
    vqe_res = minimize(vqe, params, args=(my_dict, pulse_backend, gate_backend, n_qubit, n_shot),
                       method=optimizer, constraints=LC,options={'rhobeg':0.1,'maxiter':0})
    #print('The optimized loss func value: {}'.format(vqe_res.fun))
    pvqe_energies_4.append(vqe_res.fun+ REPULSION_ENERGYproblem)
    err = abs(12/100 * vqe_res.fun)
    errore.append(err)
    print('distanza:',dist)
    print('pulse_VQE:',vqe_res.fun + REPULSION_ENERGYproblem,'+/-',err)
    print('esatto:',result.total_energies[0].real)'''

In [ ]:
import matplotlib.pyplot as plt
plt.plot(distances, pvqe_energies_1, marker = "*", linewidth=1.0, label="pulse ansatz VQE Energy")
#plt.plot(distances, pvqe_energies_2, marker = "*", linewidth=0.5, label="pulse ansatz VQE Energy")
#plt.plot(distances, pvqe_energies_3, marker = "*", linewidth=0.5, label="pulse ansatz VQE Energy")
#plt.plot(distances, pvqe_energies_4, marker = "*", linewidth=2.0, label="pulse ansatz VQE Energy")
plt.plot(distances, exact_energies, marker = "o", linewidth=0.5, label="Energy")
plt.xlabel("Atomic distance (Angstrom)")
plt.ylabel("Energy")
plt.legend()
plt.title('Energia vs Distanza H2 (2qubit-ibm_osaka)')
output_path = 'EnergyVSdistH2(2qubit-ibm_osaka).png'
plt.savefig(output_path)
plt.show()